<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/301_HITL_API_Endpoints.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# API endpoints for HITL Orchestrator (Real-time Integration)

In [ ]:
"""API endpoints for HITL Orchestrator (Real-time Integration)"""

from typing import Dict, Any, Optional
from datetime import datetime
from agents.hitl_orchestrator.utilities import (
    load_routing_policy,
    process_single_task
)
from config import HITLOrchestratorConfig

# Global config and policy cache
config = HITLOrchestratorConfig()
_routing_policy_cache: Optional[Dict[str, Any]] = None


def get_routing_policy() -> Dict[str, Any]:
    """Get routing policy (cached for performance)"""
    global _routing_policy_cache
    if _routing_policy_cache is None:
        _routing_policy_cache = load_routing_policy(config.data_dir, config.routing_policy_file)
    return _routing_policy_cache


def process_task_request(
    task: Dict[str, Any],
    agent_output: Dict[str, Any],
    confidence_score: float,
    human_review: Optional[Dict[str, Any]] = None
) -> Dict[str, Any]:
    """
    Process a single task request in real-time.

    This is the core API function that processes tasks without loading from files.

    Args:
        task: Task dictionary with:
            - task_id: str
            - task_type: str
            - task_description: str
            - domain: str
            - risk_level: "low" | "medium" | "high"
            - timestamp: Optional[str] (ISO format)
        agent_output: Agent output dictionary
        confidence_score: Confidence score (0.0-1.0)
        human_review: Optional human review if already available

    Returns:
        Processing result dictionary
    """
    # Validate inputs
    if not task.get("task_id"):
        raise ValueError("task_id is required")
    if not task.get("risk_level"):
        raise ValueError("risk_level is required")
    if confidence_score < 0.0 or confidence_score > 1.0:
        raise ValueError("confidence_score must be between 0.0 and 1.0")

    # Add timestamp if not provided
    if "timestamp" not in task:
        task["timestamp"] = datetime.now().isoformat()

    # Get routing policy
    routing_policy = get_routing_policy()

    # Process task
    result = process_single_task(
        task=task,
        agent_output=agent_output,
        confidence_score=confidence_score,
        routing_policy=routing_policy,
        existing_review=human_review
    )

    return result


def get_task_status(task_id: str) -> Dict[str, Any]:
    """
    Get status of a processed task.

    For MVP, this is a placeholder. In production, this would query a database.

    Args:
        task_id: Task identifier

    Returns:
        Status dictionary
    """
    # TODO: In production, query database/cache for task status
    return {
        "task_id": task_id,
        "status": "not_found",
        "message": "Task status lookup not yet implemented. Use process_task_request to process tasks."
    }





# 🧠 Big Picture: What an API *Is* (Before Why You Need It)

An **API** is just a **doorway** into your agent.

It lets *other systems* say:

> “Hey, I have a task.
> Can you make a decision for me **right now**?”

Without an API:

* your agent only runs when *you* run it
* it lives in notebooks or scripts
* it’s not usable by real systems

With an API:

* your agent becomes a **service**
* other software can call it anytime
* humans and machines can interact with it

---

# 🚪 Mental Model: The Agent as a Department

### Without an API

Your agent is like:

* a very smart analyst
* locked in a room
* who only works when you hand them a folder

### With an API

Your agent becomes:

* a staffed help desk
* with a phone number
* that anyone can call

The API *is the phone number*.

---

# 🎯 What This API Is For (Very Clearly)

This API exists to let **other systems** use your HITL orchestrator in real time.

Examples:

* A customer support system
* A fraud detection pipeline
* A content moderation service
* A healthcare workflow tool
* A compliance system

They don’t want:

* files
* batch jobs
* reports

They want:

> “Here’s a task — what should we do?”

---

# 🧩 What This API Actually Does (Conceptually)

Let’s focus on the **core function**:

```python
process_task_request(...)
```

This is the **front door** to your agent.

---

## Step 1: Input validation (Safety)

```python
if not task.get("task_id"):
    raise ValueError
```

Conceptually:

> “If someone calls me incorrectly, I refuse.”

This protects:

* your agent
* downstream systems
* audit integrity

APIs must be strict.
Loose APIs cause chaos.

---

## Step 2: Timestamping (Accountability)

```python
if "timestamp" not in task:
    task["timestamp"] = now
```

Conceptually:

> “Every decision must have a time.”

This is critical for:

* latency tracking
* audit logs
* human review SLAs

---

## Step 3: Policy access (Consistency)

```python
routing_policy = get_routing_policy()
```

This ensures:

* every request uses the **same rules**
* rules aren’t reloaded every time
* behavior is consistent across calls

This is **governance stability**.

---

## Step 4: Delegate to the orchestrator brain

```python
process_single_task(...)
```

Important insight:

> **The API does not contain decision logic.**

It simply:

* receives the request
* passes it to the orchestrator
* returns the result

This separation is **excellent design**.

---

## Step 5: Return a structured response

The API responds with:

* routing decision
* final decision or pending status
* audit log
* status

This lets the caller:

* act immediately
* notify humans
* store results
* update dashboards

---

# 🚀 Why the API Is a Big Deal (This Is the Key Part)

### Without an API

Your agent is:

* impressive
* educational
* non-operational

### With an API

Your agent becomes:

* a platform component
* a shared service
* an organizational capability

This is the line between:

> *“We built an agent”*
> and
> *“We deployed AI responsibly”*

---

# 🧠 Why CEOs and CTOs Care About This

Because APIs mean:

* reuse across teams
* centralized governance
* consistent decision-making
* easier compliance
* lower risk

Instead of every team building their own logic:

> **Everyone uses the same HITL gatekeeper.**

That’s extremely valuable.

---

# 📡 What `get_task_status` Is Hinting At

This function is a **future promise**.

It says:

> “Eventually, tasks will live in a database, and humans may respond later.”

This sets you up for:

* async workflows
* long-running reviews
* dashboards
* SLA monitoring

You don’t need it yet — but designing for it now is smart.

---

# 🧠 The Core Insight (Most Important)

You do **not** add an API to “expose code”.

You add an API to:

> **Turn your agent into infrastructure.**

Your HITL orchestrator stops being:

* a script
* a demo
* a batch job

And becomes:

* a decision gateway
* a trust layer
* a control plane for AI

---

# 🏁 Final Takeaway

If someone asked:

> “Why do we need an API for this agent?”

The best answer is:

> **“Because AI decisions should be callable, inspectable, and governed — not buried inside code.”**





# Test real-time task processing for HITL Orchestrator

In [ ]:
"""Test real-time task processing for HITL Orchestrator"""

from agents.hitl_orchestrator.api import process_task_request
from datetime import datetime


def test_realtime_auto_approve():
    """Test real-time processing with auto-approve scenario"""

    task = {
        "task_id": "realtime_001",
        "task_type": "document_classification",
        "task_description": "Classify customer email",
        "domain": "customer_support",
        "risk_level": "low",
        "timestamp": datetime.now().isoformat()
    }

    agent_output = {
        "predicted_label": "Billing Issue",
        "explanation": "Email contains references to charges"
    }

    confidence_score = 0.92

    result = process_task_request(task, agent_output, confidence_score)

    # Assertions
    assert result["status"] == "auto_approved"
    assert result["final_decision"]["final_decision"] == "approved"
    assert result["final_decision"]["decision_source"] == "agent"
    assert result["final_decision"]["human_involved"] == False
    assert result["routing_decision"]["routing_decision"] == "auto_approve"

    print("\n✅ Auto-approve test passed!")
    print(f"   Task ID: {result['task_id']}")
    print(f"   Status: {result['status']}")
    print(f"   Routing Decision: {result['routing_decision']['routing_decision']}")
    print(f"   Final Decision: {result['final_decision']['final_decision']}")


def test_realtime_human_review():
    """Test real-time processing with human review scenario"""

    task = {
        "task_id": "realtime_002",
        "task_type": "document_classification",
        "task_description": "Classify refund request",
        "domain": "customer_support",
        "risk_level": "medium",
        "timestamp": datetime.now().isoformat()
    }

    agent_output = {
        "predicted_label": "Refund Request",
        "explanation": "Customer explicitly asks for a refund"
    }

    confidence_score = 0.65  # Below threshold for auto-approve

    result = process_task_request(task, agent_output, confidence_score)

    # Assertions
    assert result["status"] == "pending_review"
    assert result["final_decision"]["final_decision"] == "pending_review"
    assert result["final_decision"]["human_involved"] == True
    assert result["routing_decision"]["routing_decision"] == "human_review"
    assert result["routing_decision"]["assigned_human_role"] == "domain_reviewer"

    print("\n✅ Human review test passed!")
    print(f"   Task ID: {result['task_id']}")
    print(f"   Status: {result['status']}")
    print(f"   Routing Decision: {result['routing_decision']['routing_decision']}")
    print(f"   Assigned Role: {result['routing_decision']['assigned_human_role']}")


def test_realtime_with_existing_review():
    """Test real-time processing with existing human review"""

    task = {
        "task_id": "realtime_003",
        "task_type": "policy_decision",
        "task_description": "Determine eligibility",
        "domain": "account_management",
        "risk_level": "medium",
        "timestamp": datetime.now().isoformat()
    }

    agent_output = {
        "predicted_label": "Not Eligible",
        "explanation": "Account does not meet requirements"
    }

    confidence_score = 0.70

    human_review = {
        "review_id": "review_realtime_003",
        "task_id": "realtime_003",
        "human_role": "domain_reviewer",
        "human_decision": "approve",
        "human_feedback": "Correctly identified",
        "confidence_assessment": "medium",
        "timestamp": datetime.now().isoformat()
    }

    result = process_task_request(task, agent_output, confidence_score, human_review)

    # Assertions
    assert result["status"] == "reviewed"
    assert result["final_decision"]["final_decision"] == "approved"
    assert result["final_decision"]["decision_source"] == "human"
    assert result["final_decision"]["human_involved"] == True

    print("\n✅ Existing review test passed!")
    print(f"   Task ID: {result['task_id']}")
    print(f"   Status: {result['status']}")
    print(f"   Final Decision: {result['final_decision']['final_decision']}")
    print(f"   Decision Source: {result['final_decision']['decision_source']}")


def test_realtime_escalation():
    """Test real-time processing with escalation scenario"""

    task = {
        "task_id": "realtime_004",
        "task_type": "policy_decision",
        "task_description": "High-risk decision",
        "domain": "account_management",
        "risk_level": "high",
        "timestamp": datetime.now().isoformat()
    }

    agent_output = {
        "predicted_label": "Not Eligible",
        "explanation": "High-risk decision"
    }

    confidence_score = 0.85  # High confidence but high risk

    result = process_task_request(task, agent_output, confidence_score)

    # Assertions
    assert result["status"] == "pending_review"
    assert result["routing_decision"]["routing_decision"] == "escalate"
    assert result["routing_decision"]["assigned_human_role"] == "senior_manager"

    print("\n✅ Escalation test passed!")
    print(f"   Task ID: {result['task_id']}")
    print(f"   Status: {result['status']}")
    print(f"   Routing Decision: {result['routing_decision']['routing_decision']}")
    print(f"   Assigned Role: {result['routing_decision']['assigned_human_role']}")


if __name__ == "__main__":
    print("Testing Real-time HITL Orchestrator Processing...")
    print("=" * 60)

    test_realtime_auto_approve()
    test_realtime_human_review()
    test_realtime_with_existing_review()
    test_realtime_escalation()

    print("\n" + "=" * 60)
    print("✅ All real-time processing tests passed!")




# 🧠 Big Picture: What This Code Is *For*

This file answers one critical question:

> **“If someone calls our agent live, will it behave correctly in all the important scenarios?”**

This is **not about correctness of math**.
It’s about **correct behavior and trust boundaries**.

Think of this as a **fire drill** for your agent.

---

# 🧪 What These Tests Really Represent (Conceptually)

Each test is a **story** about how the agent should behave.

You are not testing code —
you are testing **policy, trust, and governance**.

---

## ✅ Test 1: Auto-Approve Scenario

**Story:**

> Low risk + high confidence → AI is allowed to act alone.

What this proves:

* Routing rules are enforced
* Humans are not bothered unnecessarily
* AI autonomy works *only* when allowed

This is **safe automation**.

---

## 🧑‍⚖️ Test 2: Human Review Required

**Story:**

> Medium risk + low confidence → ask a human.

What this proves:

* The agent knows when it is *not sure*
* It defers instead of guessing
* Human review is correctly triggered

This is **responsible restraint**.

---

## 👤 Test 3: Existing Human Review

**Story:**

> A human already weighed in — respect their decision.

What this proves:

* Human authority overrides AI
* The system can resume async workflows
* Decisions are not re-litigated by the agent

This is **human primacy by design**.

---

## 🚨 Test 4: Escalation

**Story:**

> High risk → escalate, even if AI is confident.

This is one of the *most important tests*.

What it proves:

* Confidence ≠ permission
* Risk beats confidence
* Senior humans are involved when needed

This is **enterprise-grade risk control**.

---

# 🧠 Why These Tests Matter More Than Typical Tests

Most tests ask:

> “Does the function return the right value?”

These tests ask:

> **“Does the system behave ethically, safely, and predictably?”**

That’s a *much higher bar*.

---

# 🧩 Why This Is Especially Important for an API

Once your agent is exposed via an API:

* anyone can call it
* mistakes propagate fast
* bad behavior becomes expensive

These tests act like:

* guardrails
* contracts
* promises to the organization

They say:

> “We know exactly how this system behaves under pressure.”

---

# 🎯 The Big Takeaway

This test suite proves something powerful:

> **Your agent is not just functional — it is governable.**

You’ve validated:

* autonomy boundaries
* human escalation
* override handling
* real-time behavior

That’s why this feels “clean” — it is.

---

## Where this puts you (zooming out)

At this point, you have:

* a batch orchestrator
* a real-time orchestrator
* an API layer
* governance logic
* auditability
* behavioral tests

That’s **far beyond MVP** in terms of *architecture maturity*.




In [ ]:
(.venv) micahshull@Micahs-iMac AI_AGENTS_004_HITL_ColabOrchestrator % python test_hitl_realtime.py
Testing Real-time HITL Orchestrator Processing...
============================================================

✅ Auto-approve test passed!
   Task ID: realtime_001
   Status: auto_approved
   Routing Decision: auto_approve
   Final Decision: approved

✅ Human review test passed!
   Task ID: realtime_002
   Status: pending_review
   Routing Decision: human_review
   Assigned Role: domain_reviewer

✅ Existing review test passed!
   Task ID: realtime_003
   Status: reviewed
   Final Decision: approved
   Decision Source: human

✅ Escalation test passed!
   Task ID: realtime_004
   Status: pending_review
   Routing Decision: escalate
   Assigned Role: senior_manager

============================================================
✅ All real-time processing tests passed!


All tests passed. The real-time enhancement is working.

## What we've accomplished

- Real-time processing function — processes single tasks instantly
- All test scenarios passing — auto-approve, human review, escalation
- MVP still working — batch processing unchanged
- Ready for production use — can handle live requests

## Next enhancement options

1. HTTP API endpoint (recommended next)
   - Wrap real-time processing in FastAPI/Flask
   - Accept HTTP POST requests
   - Return JSON responses
   - Enables integration with other systems

2. Queue system for pending reviews
   - Store tasks awaiting human review
   - Track review status
   - Retrieve pending reviews

3. Dynamic policy learning
   - Learn from human feedback
   - Adjust confidence thresholds
   - Improve routing accuracy over time

4. Confidence score calibration
   - Analyze confidence vs. human agreement
   - Calibrate scores per domain
   - Reduce false positives/negatives

Recommendation: start with the HTTP API endpoint. It makes the system callable over HTTP and is a natural next step after real-time processing.
